In [ ]:
import cv2
import numpy as np

def extract_sift_features(image):
    """
    SIFT 특징점과 디스크립터를 추출하는 함수
    :param image: 입력 영상 (그레이스케일)
    :return: 키포인트, 디스크립터
    """
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2, ratio=0.75):
    """
    두 영상의 SIFT 디스크립터를 매칭하는 함수
    :param descriptors1: 첫 번째 영상의 디스크립터
    :param descriptors2: 두 번째 영상의 디스크립터
    :param ratio: Lowe의 비율 테스트 임계값
    :return: 매칭된 점 쌍
    """
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < ratio * n.distance:
            good_matches.append(m)
    
    return good_matches

def estimate_homography(keypoints1, keypoints2, matches, threshold=5.0):
    """
    RANSAC을 사용하여 호모그래피 행렬을 추정하는 함수
    :param keypoints1: 첫 번째 영상의 키포인트
    :param keypoints2: 두 번째 영상의 키포인트
    :param matches: 매칭된 점 쌍
    :param threshold: RANSAC 임계값
    :return: 호모그래피 행렬, 인라이어 마스크
    """
    if len(matches) < 4:
        return None, None
    
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, threshold)
    return H, mask

def stitch_images(images):
    """
    여러 영상을 파노라마로 합치는 함수
    :param images: 입력 영상 리스트
    :return: 파노라마 영상
    """
    if len(images) < 2:
        return None
    
    # 첫 번째 영상을 기준으로 설정
    result = images[0].copy()
    h, w = result.shape[:2]
    
    for i in range(1, len(images)):
        # 현재 영상과 다음 영상
        img1 = result
        img2 = images[i]
        
        # 그레이스케일로 변환
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        
        # SIFT 특징점 추출
        keypoints1, descriptors1 = extract_sift_features(gray1)
        keypoints2, descriptors2 = extract_sift_features(gray2)
        
        # 특징점 매칭
        matches = match_features(descriptors1, descriptors2)
        
        # 호모그래피 행렬 추정 (RANSAC 사용)
        H, mask = estimate_homography(keypoints1, keypoints2, matches)
        if H is None:
            print(f"호모그래피 추정 실패: 영상 {i}와 {i+1}")
            return None
        
        # 다음 영상을 현재 결과에 워핑
        h2, w2 = img2.shape[:2]
        warped_img2 = cv2.warpPerspective(img2, H, (w + w2, max(h, h2)))
        
        # 결과 영상의 크기 업데이트
        h, w = warped_img2.shape[:2]
        
        # img1을 warped_img2와 동일한 크기로 패딩
        padded_img1 = np.zeros_like(warped_img2)
        padded_img1[:img1.shape[0], :img1.shape[1]] = img1
        
        # 마스크 생성
        mask = np.zeros((h, w), dtype=np.uint8)
        mask[:img1.shape[0], :img1.shape[1]] = 255
        
        # 결과 영상 생성
        result = np.zeros_like(warped_img2)
        result[mask > 0] = padded_img1[mask > 0]  # 패딩된 img1을 복사
        mask = (warped_img2 > 0).astype(np.uint8) * 255
        result[mask > 0] = warped_img2[mask > 0]  # warped_img2를 복사
    
    return result

# 테스트
if __name__ == "__main__":
    # 영상 로드 (예: image1.jpg, image2.jpg, image3.jpg)
    images = [
        cv2.imread("./image/IMG_0720.png"),
        cv2.imread("./image/IMG_0721.png"),
        cv2.imread("./image/IMG_0722.png")
    ]
    
    # 영상이 제대로 로드되었는지 확인
    if any(img is None for img in images):
        print("영상을 로드할 수 없습니다. 파일 경로를 확인하세요.")
    else:
        # 파노라마 영상 제작
        panorama = stitch_images(images)
        if panorama is not None:
            # 결과 저장 및 표시
            cv2.imwrite("panorama.jpg", panorama)
            cv2.imshow("Panorama", panorama)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print("파노라마 영상 제작에 실패했습니다.")

IndexError: boolean index did not match indexed array along dimension 1; dimension is 3024 but corresponding boolean dimension is 6048